In [1]:
import os

In [2]:
from google.colab import userdata
hf_token=userdata.get('HUGGINGFACEHUB_ACCESS_TOKEN')

In [4]:
# ai tool agent

In [5]:
!pip install langchain langchain-core langchain-huggingface huggingface-hub

In [6]:
!pip install langchain_community langchain_experimental

In [7]:
from langchain_huggingface import HuggingFaceEndpoint,ChatHuggingFace
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.tools import tool




In [8]:
from langchain_classic.agents import create_react_agent,AgentExecutor

In [9]:
llm=HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-genration",
    huggingfacehub_api_token=hf_token,
)

In [10]:
model=ChatHuggingFace(llm=llm)

In [11]:
!pip install -U ddgs

In [12]:
serach_tool=DuckDuckGoSearchRun()
result=serach_tool.run("what is the capital of china")

In [13]:
print(result)

What is the Capital of China ? ... What Is The Capital Of The Shanxi Province Of China ? ... What Is The Capital Of The Yunnan Province Of China ? What Is The Capital Of The Yunnan Province Of China ? ... What Is The Capital Of The Shanxi Province Of China ? ... What Is The Capital Of The Shandong ... Jinan is the provincial capital of Shandong, and the name Jinan is derived from the name Ji which refers to the Ji River that was flowing near the ... What is the Capital of China ? ... Beijing , also known as Peking, is the capital of China and one of the most populous cities in the world. Find out what is the capital city of countries around the world. ... The Capital City of China (officially named People s Republic of China ) is the ...


In [14]:
import requests

In [15]:
# weather data
@tool
def get_current_weather(city:str)->str:
  """ these function give the data of weather """
  url=f'https://api.weatherstack.com/current?access_key=ec29ba055b3212c8dfe073390e8ad541&query={city}'
  response=requests.get(url)
  data=response.json()
  return data

In [16]:
response=get_current_weather.invoke({'city':'mumbai'})

In [17]:
!pip install -U langchainhub


In [21]:
import langchainhub

In [25]:
from langchain_classic import hub

In [26]:
prompts=hub.pull("hwchase17/react")

In [30]:
# create agent
agent=create_react_agent(
    llm=model,
    tools=[serach_tool,get_current_weather],
    prompt=prompts

)

In [31]:
# agent executor
agent_executor=AgentExecutor(
    agent=agent,
    tools=[serach_tool,get_current_weather],
    verbose=True
)

In [32]:
result=agent_executor.invoke({'input':'what is the capital of france'})



> Entering new AgentExecutor chain...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


I know that the capital of France is Paris, but I will use the duckduckgo_search tool to confirm this information.
Action: duckduckgo_search
Action Input: capital of france
ObservOctober 27, 2025 - The Paris Observatory (French: Observatoire de Paris, pronounced [ɔbsɛʁvatwaʁ d(ə) paʁi]), a research institution of the Paris Sciences et Lettres University, is the foremost astronomical observatory of France, and one of the largest astronomical centres in the world. September 19, 2025 - The 14th arrondissement of Paris ... named arrondissement de l'Observatoire (IPA: [aʁɔ̃dismɑ̃ də lɔpsɛʁvatwaʁ]; meaning "arrondissement of the Observatory"; named after the Paris Observatory ), is one of the ... June 7, 2025 - The Haute-Provence Observatory (OHP, French: Observatoire de Haute-Provence) is an astronomical observatory in the southeast of France, about 90 km east of Avignon and 100 km north of Marseille. June 18, 2025 - The Astronomy Tower of the Sorbonne is a tower at the Sorbonne University'

In [39]:
result=agent_executor.invoke({'input':"what is the weather condition of today in mumbai"})



> Entering new AgentExecutor chain...
I need to check the current weather in Mumbai.
Action: get_current_weather
Action Input: city="Mumbai"
Observ{'request': {'type': 'City', 'query': 'Mumbai, India', 'language': 'en', 'unit': 'm'}, 'location': {'name': 'Mumbai', 'country': 'India', 'region': 'Maharashtra', 'lat': '18.975', 'lon': '72.826', 'timezone_id': 'Asia/Kolkata', 'localtime': '2025-12-31 19:09', 'localtime_epoch': 1767208140, 'utc_offset': '5.50'}, 'current': {'observation_time': '01:39 PM', 'temperature': 26, 'weather_code': 143, 'weather_icons': ['https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0006_mist.png'], 'weather_descriptions': ['Haze'], 'astro': {'sunrise': '07:12 AM', 'sunset': '06:12 PM', 'moonrise': '03:00 PM', 'moonset': '03:40 AM', 'moon_phase': 'Waxing Gibbous', 'moon_illumination': 84}, 'air_quality': {'co': '283.85', 'no2': '6.45', 'o3': '217', 'so2': '40.35', 'pm2_5': '54.45', 'pm10': '57.35', 'us-epa-index': '3', 'gb-defra-index': '3'}

In [40]:
print(result['output'])

Today in Mumbai, the weather condition is haze with a temperature of 26°C, wind speed at 17 km/h coming from the northwest direction, and humidity at 65%. It's nighttime currently, and there's no precipitation expected. The air quality is generally good but watch out for CO and PM2.5 levels.
